### Whisp a feature collection

Import packages
- NB to avoid complications from altering main python environment run using a virtual environment (https://docs.python.org/3/tutorial/venv.html)

In [1]:
import ee

import pandas as pd

from pathlib import Path

import whisp

ee.Authenticate()
ee.Initialize() 

EE auto-initialized with default credentials.


Get a feature collection

In [2]:
GEOJSON_EXAMPLE_FILEPATH = (
    Path.cwd().parents[0]/ "test" / "fixtures" / "geojson_example.geojson"
)
print(GEOJSON_EXAMPLE_FILEPATH)


c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\fixtures\geojson_example.geojson


Whisp it

In [3]:
df_formatted_stats = whisp.whisp_formatted_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH,external_id_column=None)

Received geojson_filepath: c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\fixtures\geojson_example.geojson
Reading GeoJSON file from: c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\fixtures\geojson_example.geojson
using valid datasets filter due to error in input
Invalid image: Image.remap: Image has no bands.
Invalid image: Image.lt: If one image has no bands, the other must also have no bands. Got 0 and 1.
File C:\Users\Arnell\Documents\GitHub\whisp_sustaain\whisp\parameters\lookup_gee_datasets.csv changed, updating schema...
File C:\Users\Arnell\Documents\GitHub\whisp_sustaain\whisp\parameters\lookup_context_and_metadata.csv changed, updating schema...
Creating or updating schema based on changed files...
Schema DataFrame columns: Index(['name', 'order', 'theme', 'use_for_risk', 'exclude_from_output',
       'col_type', 'is_nullable', 'is_required', 'corresponding_variable',
       'exclude_from_input'],
      dtype='object')
Processing column: plotId, Type: string, Null

Display table

In [9]:
df_formatted_stats #view output dataframe

,plotId,external_id,Plot_area_ha,Geometry_type,Country,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,In_waterbody,...,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,RADD_before_2020,TMF_deg_after_2020,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,DIST_after_2020
0,1,NaN,1.939000,Polygon,GHA,Ashanti Region,-1.611942,6.159540,ha,False,...,1.552000,0.000000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.00
1,2,NaN,4.152000,Polygon,GHA,Ashanti Region,-1.644732,6.104735,ha,False,...,2.724000,0.000000,0.000,0.00,0.000,0.089,0.743,0.000,0.000,0.00
2,3,NaN,16.600000,Polygon,GHA,Western Region,-2.157144,5.981149,ha,False,...,0.000000,0.000000,0.000,1.11,0.089,0.000,0.000,0.000,1.121,0.00
3,4,NaN,31.212999,Polygon,IDN,South Sumatra,103.956096,-3.054668,ha,False,...,31.021999,31.212999,6.403,0.00,0.000,0.000,0.000,0.000,0.000,0.00
4,5,NaN,1.964000,Polygon,IDN,South Sumatra,103.970371,-3.068831,ha,False,...,1.950000,0.000000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.00
5,6,NaN,12.725000,Polygon,IDN,South Sumatra,103.975182,-3.082922,ha,False,...,1.899000,0.000000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.00
6,7,NaN,20.882000,Polygon,IDN,South Sumatra,103.977512,-3.083808,ha,False,...,9.043000,0.000000,0.000,0.00,0.714,0.000,0.000,0.000,0.000,0.00
7,8,NaN,8.279000,Polygon,CIV,Lagunes,-4.101646,5.711935,ha,False,...,3.194000,0.000000,0.000,0.00,0.000,0.000,1.602,0.000,0.001,0.00
8,9,NaN,1.981000,Polygon,CIV,Lagunes,-4.086848,5.673811,ha,False,...,0.435000,0.000000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.00
9,10,NaN,3.797000,Polygon,CIV,District Autonome D'Abidjan,-4.119589,5.572136,ha,False,...,1.161000,0.000000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.00


Export csv

In [8]:
# Define output directory
out_directory = Path.cwd().parents[0] / "test" / "output" # directory untracked by git

# Define output file path
output_stats_file = out_directory / "whisp_output_table.csv" # edit as required

# Save statistics to CSV
df_formatted_stats.to_csv(path_or_buf=output_stats_file,index=False)

print(f"Formatted stats saved to: {output_stats_file}")


Formatted stats saved to: c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\output\whisp_output_table.csv


Calculate risk category

In [6]:
# add risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(df=df_formatted_stats)

C:\Users\Arnell\Documents\GitHub\whisp_sustaain\whisp\src\risk.py:223: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = low_name
C:\Users\Arnell\Documents\GitHub\whisp_sustaain\whisp\src\risk.py:223: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = low_name
C:\Users\Arnell\Documents\GitHub\whisp_sustaain\whisp\src\risk.py:223: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

Display table with risk columns

In [7]:
df_w_risk

,plotId,external_id,Plot_area_ha,Geometry_type,Country,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,In_waterbody,...,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,DIST_after_2020,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,1,NaN,0.01,Point,COD,Kwango,18.974532,-6.575983,ha,False,...,0.0,0.0,0.01,0.0,0.0,yes,no,yes,yes,low
1,2,NaN,0.01,Point,COD,Kinshasa,16.273743,-4.555583,ha,False,...,0.0,0.0,0.01,0.0,0.0,yes,no,yes,yes,low
2,3,NaN,0.01,Point,COD,Sud-Ubangi,19.006602,3.556204,ha,False,...,0.0,0.0,0.01,0.0,0.0,yes,no,yes,yes,low
3,4,NaN,0.01,Point,COD,Lomami,24.411313,-5.528367,ha,False,...,0.0,0.0,0.00,0.0,0.0,yes,no,yes,no,low
4,5,NaN,0.01,Point,COD,Kasai,20.295683,-6.128892,ha,False,...,0.0,0.0,0.01,0.0,0.0,yes,no,yes,yes,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,NaN,0.01,Point,COD,Tshuapa,20.017114,-1.782481,ha,False,...,0.0,0.0,0.00,0.0,0.0,yes,no,no,no,more_info_needed
96,97,NaN,0.01,Point,COD,Tshopo,25.846912,-0.424859,ha,False,...,0.0,0.0,0.00,0.0,0.0,yes,no,no,no,more_info_needed
97,98,NaN,0.01,Point,COD,Sud-Ubangi,18.178444,2.025029,ha,False,...,0.0,0.0,0.00,0.0,0.0,yes,no,no,no,more_info_needed
98,99,NaN,0.01,Point,COD,Équateur,19.132362,0.636590,ha,False,...,0.0,0.0,0.00,0.0,0.0,yes,no,no,no,more_info_needed


Export table with risk columns to csv 

In [9]:
# Define output file path
output_risk_file = out_directory / "whisp_output_table_w_risk.csv" # edit as required

# Save statistics with added risk columns to CSV
df_w_risk.to_csv(path_or_buf=output_risk_file,index=False)

print(f"Table with risk columns saved to: {output_risk_file}")

Table with risk columns saved to: c:\Users\Arnell\Documents\GitHub\whisp_sustaain\test\output\whisp_output_table_w_risk.csv
